In [1]:
import math
import nbimporter
from Brick_Element_Stiffness_Matrix import Element_K
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import spsolve
from matplotlib import pyplot as plt

In [2]:
class FEM_3D():
    def __init__(self, nodes_x, nodes_y, nodes_z, filter_R, nu):
        self.nx = nodes_x
        self.ny = nodes_y
        self.nz = nodes_z
        self.ex = nodes_x - 1
        self.ey = nodes_y - 1
        self.ez = nodes_z - 1
        
        self.n_nodes = nodes_x * nodes_y * nodes_z
        self.n_elems = self.ex * self.ey * self.ez
        
        self.n_dofs = 3 * self.n_nodes
        self.all_dofs = np.arange(self.n_dofs)
        self.fixed_dofs = np.array([], np.int32)
        self.dofs_matrix = np.zeros((self.n_elems, 24), np.int32)
        
        self.K = None
        self.F = np.zeros(self.n_dofs)
        self.U = np.zeros(self.n_dofs)
        
        self.nu = nu
        self.filter_R = filter_R
    
    # Calculates DOFs indexes for element given coords of its pivot node
    def elem_DOFs(self, px, py, pz):
        # Nodes are ordered in uniform numeration
        node_1 = (px, py, pz)
        node_2 = (px - 1, py, pz)
        node_3 = (px - 1, py + 1, pz)
        node_4 = (px, py + 1, pz)
        node_5 = (px, py, pz - 1)
        node_6 = (px - 1, py, pz - 1)
        node_7 = (px - 1, py + 1, pz - 1)
        node_8 = (px, py + 1, pz - 1)
        nodes = [node_1, node_2, node_3, node_4, node_5, node_6, node_7, node_8]
        
        node_index = lambda coords: ((self.ny*self.nz)*coords[0] + self.nz*coords[1] + coords[2])
        
        nodes_indexes = np.array([node_index(node) for node in nodes])
        kron_tmp = np.ones(3) # 3 Degrees of freedom
        
        element_dofs = np.kron(nodes_indexes, kron_tmp)*3
        element_dofs[1::3] += 1
        element_dofs[2::3] += 2
        
        return element_dofs
    
    # Calculates DOFs for each element
    def form_elements(self):
        slice_elems = self.ez * self.ey
        for ex_ in range(self.ex):
            for ey_ in range(self.ey):
                for ez_ in range(self.ez):
                    ei = slice_elems*ex_ + self.ez*ey_ + ez_
                    pivot_node_x = ex_ + 1
                    pivot_node_y = ey_
                    pivot_node_z = ez_ + 1
                    element_dofs = self.elem_DOFs(pivot_node_x, pivot_node_y, pivot_node_z)
                    self.dofs_matrix[ei] = element_dofs
                    
    def assemble_sparse_K(self, dens, penal, E_void, E0):
        local_K_elems = 24*24
        
        global_iK = np.zeros(local_K_elems * self.n_elems)
        global_jK = np.zeros(local_K_elems * self.n_elems)
        global_vK = np.zeros(local_K_elems * self.n_elems)
        
        for ei in range(self.dofs_matrix.shape[0]):
            elem_density = dens[ei]
            
            young_modulus = E_void + pow(elem_density, penal) * (E0 - E_void)
            KE = Element_K().element_K(young_modulus, self.nu)
            
            iK = np.kron(self.dofs_matrix[ei], np.ones(24))
            jK = np.reshape(np.kron(self.dofs_matrix[ei], np.reshape(np.ones(24), (24,1))), -1)
            val_K = np.reshape(KE, -1)
            
            global_iK[ei*local_K_elems : (ei+1)*local_K_elems] = iK
            global_jK[ei*local_K_elems : (ei+1)*local_K_elems] = jK
            global_vK[ei*local_K_elems : (ei+1)*local_K_elems] = val_K
            
        self.K = sparse.coo_matrix((global_vK, (global_iK, global_jK)), shape = (self.n_dofs, self.n_dofs)).tocsr()
        
    def solve_U(self):
        free_dofs = np.setdiff1d(self.all_dofs, self.fixed_dofs)
        self.U[free_dofs] = spsolve(self.K[free_dofs, :][:, free_dofs], self.F[free_dofs])
        
        return self.U
    
    def fix_node(self, node_x, node_y, node_z, fix_x, fix_y, fix_z):
        node_index = (self.ny*self.nz)*node_x + self.nz*node_y + node_z
        
        dof_x = 3*node_index
        dof_y = 3*node_index + 1
        dof_z = 3*node_index + 2
        
        if (fix_x == True):
            self.fixed_dofs = np.append(self.fixed_dofs, dof_x)
        if (fix_y == True):
            self.fixed_dofs = np.append(self.fixed_dofs, dof_y)
        if (fix_z == True):
            self.fixed_dofs = np.append(self.fixed_dofs, dof_z)
            
    def apply_load(self, node_x, node_y, node_z, load_x, load_y, load_z):
        node_index = (self.ny*self.nz)*node_x + self.nz*node_y + node_z
        
        dof_x = 3*node_index
        dof_y = 3*node_index + 1
        dof_z = 3*node_index + 2
        
        self.F[dof_x] = load_x
        self.F[dof_y] = load_y
        self.F[dof_z] = load_z
        
    def H_ei(self, e1, e2):
        slice_elems = int(self.ey * self.ez)
        
        e1_slice = e1 // slice_elems
        e1_col = (e1 % slice_elems) // self.ez
        e1_row = (e1 % slice_elems) % self.ez
        
        e2_slice = e2 // slice_elems
        e2_col = (e2 % slice_elems) // self.ez
        e2_row = (e2 % slice_elems) % self.ez
        
        h_ei = max(0, self.filter_R - np.sqrt((e1_slice-e2_slice)**2 + (e1_col-e2_col)**2 + (e1_row-e2_row)**2))
        
        return h_ei
    
    def prepare_filter(self):
        H_rows = []
        slice_elems = self.ey*self.ez
        
        for ei in range(self.n_elems):
            neighbors = np.zeros(self.n_elems)
            
            e_slice = ei // slice_elems
            e_col = (ei % slice_elems) // self.ez
            e_row = (ei % slice_elems) % self.ez
            
            min_row = max(0, e_row - np.ceil(self.filter_R))
            max_row = min(self.ez, e_row + np.ceil(self.filter_R))
            min_col = max(0, e_col - np.ceil(self.filter_R))
            max_col = min(self.ey, e_col + np.ceil(self.filter_R))
            min_slice = max(0, e_slice - np.ceil(self.filter_R))
            max_slice = min(self.ex, e_slice + np.ceil(self.filter_R))
            
            for i_slice in range(min_slice, max_slice):
                for i_col in range(min_col, max_col):
                    for i_row in range(min_row, max_row):
                        elem_index = i_slice*slice_elems + self.ez*i_col + i_row
                        neighbors[elem_index] = self.H_ei(ei, elem_index)
                        
            H_rows.append(neighbors)
        
        H = np.stack(H_rows, axis = 0)
        sH = np.sum(H, axis = 1)
        
        return H, sH
        
    def save_config(self, densities, path):
        output_file = open(path, 'w')
        
        output_file.write(f'{self.nx} {self.ny} {self.nz}\n')
        output_file.write('section\n')
        
        for i in range(int(self.n_dofs / 3)):
            displ_x = self.U[3*i]
            displ_y = self.U[3*i + 1]
            displ_z = self.U[3*i + 2]
            output_file.write(f'{displ_x} {displ_y} {displ_z}\n')
        output_file.write('section\n')
        
        for i in range(densities.shape[0]):
            output_file.write(f'{densities[i]}\n')
        
        output_file.close()